## Bartosz Sroka

In [22]:
# Import opowiednich bibliotek
import pandas as pd
import numpy as np
import os

In [23]:
# Wybór odpowiedniego zadania
def choose_data(surname):
    list_of_data = ['weather.txt', 'tb.csv', 'billboard.csv', 'drinks.csv', 'earthquake_data.csv']
    print("Wybrane zadanie to:\t " + list_of_data[len(surname)%5])
choose_data("sroka")

Wybrane zadanie to:	 weather.txt


In [21]:
# Wczytanie danych 
if not os.path.exists("weather.txt"):
    !wget https://raw.githubusercontent.com/barteks019/AiBD_BartoszSroka_1b/blob/main/Lab%202%20-%2013.10.2021/OriginalData/weather.txt

In [274]:
weather = open('weather.txt', 'r')

def convert_data(data, year, number_of_months):
    converted_data = []
    while True:
        line = data.readline()
        splited_line = [x for x in line.replace('-', ' -').split(' ')[:-1] if x]
        if splited_line == []:
            break
        splited_line = [splited_line[0][:11], int(splited_line[0][11:15]), int(splited_line[0][-6:-4]), splited_line[0][-4:]] + splited_line[1:]
        if splited_line[1] == year and splited_line[2] <= number_of_months:
            filtered_data_from_validators = [x for x in splited_line[4:] if x != "I" if x != "S"]
            change_format_of_temperature_to_int = [int(x) for x in filtered_data_from_validators]
            filtered_data_from_invalid_numbers = splited_line[:4] + [x if x >= 0 else np.nan for x in change_format_of_temperature_to_int]
            converted_data.append(filtered_data_from_invalid_numbers)
    return converted_data

[print(x) for x in convert_data(weather, 2010, 5)];

['MX000017004', 2010, 1, 'TMAX', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 278, nan]
['MX000017004', 2010, 1, 'TMIN', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 145, nan]
['MX000017004', 2010, 1, 'PRCP', nan, nan, nan, nan, 0, nan, 0, nan, nan, nan, nan, 0, nan, 0, 0, nan, nan, 0, 0, 0, 0, 0, 0, nan, 0, 0, 0, 0, 0, 0, nan]
['MX000017004', 2010, 2, 'TMAX', nan, 273, 241, nan, nan, nan, nan, nan, nan, nan, 297, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 299, nan, nan, nan, nan, nan, nan, nan, nan]
['MX000017004', 2010, 2, 'TMIN', nan, 144, 144, nan, nan, nan, nan, nan, nan, nan, 134, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 107, nan, nan, nan, nan, nan, nan, nan, nan]
['MX000017004', 2010, 2, 'PRCP', nan, 0, 0, nan, nan, 0, 0, nan, 0, nan, 0, nan, 0, nan, nan, 

In [279]:
weather = open('weather.txt', 'r')
data_frame = pd.DataFrame(convert_data(weather, 2010, 5))
data_frame.columns = ["id", "year", "month", "type"] + [i for i in range(1,32)]
data_frame

,id,year,month,type,1,2,3,4,5,6,...,22,23,24,25,26,27,28,29,30,31
0,MX000017004,2010,1,TMAX,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,278.0,NaN
1,MX000017004,2010,1,TMIN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.0,NaN
2,MX000017004,2010,1,PRCP,NaN,NaN,NaN,NaN,0.0,NaN,...,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,MX000017004,2010,2,TMAX,NaN,273.0,241.0,NaN,NaN,NaN,...,NaN,299.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX000017004,2010,2,TMIN,NaN,144.0,144.0,NaN,NaN,NaN,...,NaN,107.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MX000017004,2010,2,PRCP,NaN,0.0,0.0,NaN,NaN,0.0,...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
6,MX000017004,2010,3,TMAX,NaN,NaN,NaN,NaN,321.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MX000017004,2010,3,TMIN,NaN,NaN,NaN,NaN,142.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MX000017004,2010,3,PRCP,NaN,0.0,NaN,0.0,0.0,0.0,...,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN
9,MX000017004,2010,4,TMAX,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,363.0,NaN,NaN,NaN,NaN
